In [ ]:
from IPython.display import HTML

HTML("""
<style>
    #notebook-container {
    width: 80% !important;
}
.dataTable {
    width: 100% !important;
}
.dataTables_scrollBody {
    width: 100% !important;
}
.dataTables_scrollHead {
    width: 100% !important;
}
.dataTables_filter {
   float: right !important;
}
.output_html {
    max-width: calc(100%) !important;
}
.rendered{
    font-size: 125%;
}
</style>
""")

html1 = '<img src="https://images.theconversation.com/files/45159/original/rptgtpxd-1396254731.jpg" width="900" style="margin-left:auto; margin-right:auto"/>'
HTML(html1)

# A quick analysis of my reading habits

This project aims to answer a few basic questions about my reading habits over the past few years. Specifically, it looks at the books I've read since the end of 2020, when I started keeping track of things. I've compiled a spreadsheet that contains basic information such as Title, Author, Genre, Format, etc. in the hopes of perhaps identifying trends of patterns in my reading habits. Additionally, this helps me see how my tastes change over time. Some basic questions I hope to answer are the following:

* What do I read the most?
* What format do I prefer?
* Are there any Genre-Sub-Genre pairs I tend to gravitate towards?
* Who is my most read author?
* How long does it take me to read books?
* Do I read more during a certain time of year?

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plot

df = pd.read_csv('BookList.csv')

#removes any blank rows or books I never finished/rated
df = df[df['Rating'].notnull()]

# convert columns to correct (or better) data types
df[['Year', 'Pages']] = df[['Year', 'Pages']].apply(pd.to_numeric, downcast='integer')
df['Duration'] = df['Duration'].apply(pd.to_numeric, downcast='integer')
df['Rating'] = df['Rating'].apply(pd.to_numeric, downcast='float')
df['Start Date']= pd.to_datetime(df['Start Date'])
df['Finish Date']= pd.to_datetime(df['Finish Date'])
df.tail()

### Genre Count

In [ ]:
title = "Top 5 Most-Read Genres"
sub = f"I've read almost twice as many fiction books as all other genres combined!"

plot.gen_bar_graph(df, 'Genre', title, sub, color="#d27575")

In [ ]:
title = "Top 10 Most-Read Sub-Genres"
sub = f"I've been more interested in reality-based books lately - I love learning about the history of things!"

plot.gen_bar_graph(df, 'Sub-Genre', title, sub, 10, color="#529b9c")

### Most Common Genre-Sub-Genre Pairing

In [ ]:
title = "Top 10 Most Common Genre/Sub-Genre Pairings"
sub = f"Historical Fiction really caught my interest in the past few years"

df['Genre Pair'] = df['Genre'] + '-' + df['Sub-Genre']
plot.gen_bar_graph(df, 'Genre Pair', title, sub, num=10, color="#eac392")

### Most-Read Author

In [ ]:
title = "Most-Read Authors"
sub = f"I really enjoy Japanese authors - who knew?"
plot.gen_bar_graph(df, 'Author', title, sub, color="#9cba8f")

In [ ]:
physical = (df[df['Format'] == 'Physical'].groupby('Genre')['Title']
            .count()
            .reset_index()
            .rename(columns={'Title':'Physical'}))
ebook = (df[df['Format'] == 'eBook'].groupby('Genre')['Title']
         .count()
         .reset_index()
         .rename(columns={'Title':'eBook'}))

format_df = pd.concat([physical, ebook['eBook']], axis=1).fillna(0).set_index('Genre')
format_df = format_df.convert_dtypes()

title = "Most-Read Book Format"
sub = f"I tend to buy physical copies of books based on reality and non-fiction"
plot.gen_stacked_bar_graph(format_df, title, sub)

### Ratings

In [ ]:
dfr = df.groupby('Genre')['Rating'].mean().reset_index().round(2)
dfr['Total'] = df.groupby('Genre')['Title'].count().reset_index(drop=True)

title = "Ratings by Genre"
sub = f"I tend to enjoy more cerebral books - I love intellectual stimulation!"
plot.gen_bar_graph(dfr, 'Genre', title, sub, avg=True, w_avg='Rating')

In [ ]:
dfs = df.groupby('Sub-Genre')['Rating'].mean().reset_index().round(2)
dfs['Total'] = df.groupby('Sub-Genre')['Title'].count().reset_index(drop=True)
dfs = dfs.sort_values('Rating', ascending=True).reset_index(drop=True)

title = "Ratings by Sub-Genre"
sub = f"Personal stories and books based on history tend to capture me more"
plot.gen_hbar_graph(dfs, 'Sub-Genre', title, sub, avg=True, w_avg='Rating')

### Durations

In [ ]:
dft = df.groupby('Genre')['Duration'].mean().reset_index()
dft = dft[dft['Duration'].notnull()].round(2)
dft['Total'] = df.groupby('Genre')['Title'].count().reset_index(drop=True)
# dft.loc[len(dft.index)] = ['Overall', dft['Duration'].mean()]
title = "Duration by Genre"
sub = f"Philosophy books can be incredibly difficult to get through for me"
plot.gen_bar_graph(dft, 'Genre', title, sub, avg=True, w_avg='Duration')

In [ ]:
dfy = df.groupby('Sub-Genre')['Duration'].mean().reset_index()
dfy = dfy[dfy['Duration'].notnull()].round(2)
dfy['Total'] = df.groupby('Sub-Genre')['Title'].count().reset_index(drop=True)
dfy = dfy.sort_values('Duration', ascending=True).reset_index(drop=True)

title = "Duration by Sub-Genre"
sub = f"Most non-fiction or reality-based books tend to take me longer to read"
plot.gen_hbar_graph(dfy, 'Sub-Genre', title, sub, avg=True, w_avg='Duration')

In [ ]:
#get list of unique genres/subgenres 
subgenres = df['Sub-Genre'].unique()
genres = df['Genre'].unique()

#create new, NaN-filled dataframe where rows=genres, and columns=subgenres
dfh = pd.DataFrame(np.nan, columns=subgenres, index=genres)

# grp = data grouped by the two columns you want as vertical & horizontal axes
grp = df[df["Genre"].isin(genres)].groupby(["Genre", "Sub-Genre"], as_index=False)['Title'].count()

#for each row in grp (which is the count of genre-sub-genre pairs), put the count (which is 'Title') in the corresponding row/column
for idx, row in grp.iterrows():
    dfh.loc[row["Genre"], row["Sub-Genre"]] = row['Title']
    
dfh.fillna(0, inplace=True)

title = "Genre Pairing Heatmap"
sub = f"Because who doesn't love a heatmap?"
plot.gen_heatmap(dfh, title, sub)

In [ ]:
dfc = df.groupby('Genre')['Pages'].mean().round(2).reset_index()
dfc['Total'] = df.groupby('Genre')['Title'].count().reset_index(drop=True)
dfc = dfc.sort_values('Pages', ascending=False).reset_index(drop=True)

title = "Average Number of Pages by Genre"
sub = f"Books grounded in reality need more pages to fully explain concepts and ideas"
plot.gen_bar_graph(dfc, 'Genre', title, sub, avg=True, w_avg='Pages')

In [ ]:
dfx = df.groupby('Sub-Genre')['Pages'].mean().round(2).reset_index()
dfx['Total'] = df.groupby('Sub-Genre')['Title'].count().reset_index(drop=True)
dfx = dfx.sort_values('Pages', ascending=True).reset_index(drop=True)

title = "Average Number of Pages by Sub-Genre"
sub = f"Fantasy authors love to flesh out their books"
plot.gen_hbar_graph(dfx, 'Sub-Genre', title, sub, avg=True, w_avg='Pages')

### Top Rated Books

In [ ]:
top_df = df.sort_values(by=['Rating']).tail(15).reset_index(drop=True)

title = "My Top Rated Books"
sub = f"Seriously, please read these"
plot.top10_graph(top_df, 'Rating', 'Title', title, sub)

In [ ]:
dfd = df[df['Start Date'].notnull()] #books that have a definied start (& end)

#group by year
by_start = dfd.groupby(dfd['Start Date'].dt.to_period('Y').rename('Date'))['Title'].count().rename('Started').reset_index()
by_finish = dfd.groupby(dfd['Finish Date'].dt.to_period('Y').rename('Date'))['Title'].count().rename('Finished').reset_index()
df_year = pd.merge(by_start, by_finish, how="outer").fillna(0).sort_values('Date').reset_index(drop=True)
df_year['Date'] = df_year['Date'].astype(str)
df_year = df_year.set_index('Date')

#group by year-month
by_start = dfd.groupby(dfd['Start Date'].dt.to_period('M').rename('Date'))['Title'].count().rename('Started').reset_index()
by_finish = dfd.groupby(dfd['Finish Date'].dt.to_period('M').rename('Date'))['Title'].count().rename('Finished').reset_index()
df_yearmonth = pd.merge(by_start, by_finish, how="outer").fillna(0).sort_values('Date').reset_index(drop=True)
df_yearmonth['Date'] = df_yearmonth['Date'].astype(str)
df_yearmonth = df_yearmonth.set_index('Date')

#combine both datasets so we can have a chart with a dropdown
df_time = pd.concat([df_year, df_yearmonth], axis=0)

title = "How Much I've Read Over Time"
sub = "Total Number of Books Finished by Year/Month"
plot.gen_time_graph(df_time, title, sub)

In [ ]:
# # Clean up space
#       - name: Remove unnecessary files
#         run: |
#           rm -rf /usr/share/dotnet
#           rm -rf /opt/ghc
#           rm -rf "/usr/local/share/boost"
#           rm -rf "$AGENT_TOOLSDIRECTORY"